# Guide pour l'Appel à une API Hugging Face pour la Segmentation d'Images

Bienvenue ! Ce notebook a pour but de vous guider pas à pas dans l'utilisation de l'API d'inférence de Hugging Face pour effectuer de la segmentation d'images. La segmentation d'images consiste à attribuer une étiquette (comme "cheveux", "vêtement", "arrière-plan") à chaque pixel d'une image.

Nous allons :
1. Comprendre ce qu'est une API et comment s'y connecter.
2. Envoyer une image à un modèle de segmentation hébergé sur Hugging Face.
3. Récupérer et interpréter les résultats.
4. Visualiser les masques de segmentation.
5. Étendre cela pour traiter plusieurs images.

## 1. Configuration Initiale et Importations

Commençons par importer les bibliothèques Python nécessaires. Nous aurons besoin de :
- `os` pour interagir avec le système de fichiers (lister les images).
- `requests` pour effectuer des requêtes HTTP vers l'API.
- `PIL (Pillow)` pour manipuler les images.
- `matplotlib.pyplot` pour afficher les images et les masques.
- `numpy` pour la manipulation des tableaux (les images sont des tableaux de pixels).
- `tqdm.notebook` pour afficher une barre de progression (utile pour plusieurs images).
- `base64` et `io` pour décoder les masques renvoyés par l'API.

In [ ]:
import base64
import io
import os
from pathlib import Path
import pandas as pd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import requests
from PIL import Image
from tqdm.notebook import tqdm

### Variables de Configuration

Nous devons définir quelques variables :
- `image_dir`: Le chemin vers le dossier contenant vos images. **Assurez-vous de modifier ce chemin si nécessaire.**
- `max_images`: Le nombre maximum d'images à traiter (pour ne pas surcharger l'API ou attendre trop longtemps).
- `api_token`: Votre jeton d'API Hugging Face. **IMPORTANT : Gardez ce jeton secret !**

**Comment obtenir un token API Hugging Face ?**
1. Créez un compte sur [huggingface.co](https://huggingface.co/).
2. Allez dans votre profil -> Settings -> Access Tokens.
3. Créez un nouveau token (par exemple, avec le rôle "read").
4. Copiez ce token ici.

In [ ]:
from dotenv import load_dotenv

# initialisation du dossier où sont stockées mes images
# image_dir = r"D:\ia\appli\fashiontrend\data\external\IMG" # Exemple : si vous êtes sur Colab et avez uploadé un dossier

image_dir = os.path.join(
    os.path.dirname(os.getcwd()), r"data\external\IMG"
)  # dans mon cas, image_dir = r"D:\ia\appli\fashiontrend\data\external\IMG"
max_images = 3  # Commençons avec peu d'images, os.getcwd() c'st le repertoire courant

# Charge .env et récupére mon token de HUGGING 
load_dotenv()
api_token = os.getenv("HF_TOKEN")


# Créons le dossier d'images s'il n'existe pas (pour l'exemple)
if not os.path.exists(image_dir):
    os.makedirs(image_dir)
    print(f"Dossier '{image_dir}' créé. Veuillez y ajouter des images .jpg ou .png.")
else:
    print(f"Dossier '{image_dir}' existant.")

if api_token == "VOTRE_TOKEN_HUGGING_FACE_ICI":
    print(
        "\nATTENTION : Vous devez remplacer 'VOTRE_TOKEN_HUGGING_FACE_ICI' par votre token API personnel."
    )

### Pre-Requis 2) Vérifier la connexion au compte Hugging Face

- On vérifie que le token donne bien accès à ton compte (nom / organisations).
- Si erreur : re-vérifie la ligne `HUGGINGFACEHUB_API_TOKEN` dans `.env`.

In [ ]:
from huggingface_hub import whoami

try:
    info = whoami(token=api_token)
    print("🎉 Connexion OK à Hugging Face")
    print("👤 Utilisateur :", info.get("name"))
    print("🏢 Organisations :", info.get("orgs"))
except Exception as e:
    raise SystemExit(f"❌ Erreur d'authentification : {e}") from e

## 2. Comprendre l'API d'Inférence Hugging Face

L'API d'inférence permet d'utiliser des modèles hébergés sur Hugging Face sans avoir à les télécharger ou à gérer l'infrastructure.

- **Modèle utilisé** : Nous allons utiliser le modèle `sayeed99/segformer_b3_clothes`, spécialisé dans la segmentation de vêtements et de parties du corps.
- **URL de l'API** : L'URL pour un modèle est généralement `https://api-inference.huggingface.co/models/NOM_DU_MODELE`.
- **Headers (En-têtes)** : Pour s'authentifier et spécifier le type de contenu, nous envoyons des en-têtes avec notre requête.
    - `Authorization`: Contient notre token API (précédé de `Bearer `).
    - `Content-Type`: Indique que nous envoyons une image au format JPEG (ou PNG selon le cas).

In [ ]:
API_URL = "https://api-inference.huggingface.co/models/sayeed99/segformer_b3_clothes"  # Remplacez ... par le bon endpoint.
headers = {
    "Authorization": f"Bearer {api_token}"
}  # Le "Content-Type" sera ajouté dynamiquement lors de l'envoi de l'image

## Vérifions que le token est valide en envoyant une requête GET simple
reponse = requests.get(API_URL, headers=headers)
if reponse.status_code == 200:
    print("Le token est valide !")
# print("Infos du modèle :", response.json())
elif reponse.status_code == 401:
    print("Le token est invalide ou expiré.")
else:
    print(f"Erreur inattendue : code {reponse.status_code}")

In [ ]:
# Lister les chemins des images à traiter
## On parcourt le dossier 'image_dir' et on garde uniquement les fichiers .jpg, .jpeg ou .png
valid_extens = [".jpg", ".jpeg", ".png"]  # extensions acceptées
image_paths = [
    str(p) for p in Path(image_dir).iterdir() if p.is_file() and p.suffix.lower() in valid_extens
]  # image_paths = image_paths[:max_images], si trop d’images, on limite à 'max_images'
if not image_paths:
    print(f"Aucune image trouvée dans '{image_dir}'. Veuillez y ajouter des images.")
else:
    print(f"{len(image_paths)} image(s) à traiter : {image_paths}")

In [ ]:
# Lister les chemins des masques à traiter
mask_dir = os.path.join(
    os.path.dirname(os.getcwd()), r"data\external\Mask"
)  # dans mon cas, mask_dir = r"D:\ia\appli\fashiontrend\data\external\Mask"
## On parcourt le dossier 'mask_dir' et on garde uniquement les fichiers .jpg, .jpeg ou .png
mask_paths = [
    str(p) for p in Path(mask_dir).iterdir() if p.is_file() and p.suffix.lower() in valid_extens
]

## Assurons nous d'avoir des masques dans le dossier 'mask_dir'!
if not mask_paths:
    print(f"Aucune image trouvée dans '{mask_dir}'. Veuillez y ajouter des images en masque.")
else:
    print(f"{len(mask_paths)} masque(s) à traiter : {mask_paths}")

## 3. Fonctions Utilitaires pour le Traitement des Masques

Le modèle que nous utilisons (`sayeed99/segformer_b3_clothes`) renvoie des masques pour différentes classes (cheveux, chapeau, etc.). Ces masques sont encodés en base64. Les fonctions ci-dessous sont fournies pour vous aider à :
1.  `CLASS_MAPPING`: Un dictionnaire qui associe les noms de classes (ex: "Hat") à des identifiants numériques.
2.  `get_image_dimensions`: Récupérer les dimensions d'une image.
3.  `decode_base64_mask`: Décoder un masque de base64 en une image (tableau NumPy) et le redimensionner.
4.  `create_masks`: Combiner les masques de toutes les classes détectées en un seul masque de segmentation final, où chaque pixel a la valeur de l'ID de sa classe.

**Cette partie est donnée car elle est spécifique au format de sortie de ce modèle et un peu complexe pour une première approche.** Lisez-la pour comprendre son rôle, mais ne vous attardez pas sur les détails d'implémentation pour l'instant.

In [ ]:
CLASS_MAPPING = {
    "Background": 0,
    "Hat": 1,
    "Hair": 2,
    "Sunglasses": 3,
    "Upper-clothes": 4,
    "Skirt": 5,
    "Pants": 6,
    "Dress": 7,
    "Belt": 8,
    "Left-shoe": 9,
    "Right-shoe": 10,
    "Face": 11,
    "Left-leg": 12,
    "Right-leg": 13,
    "Left-arm": 14,
    "Right-arm": 15,
    "Bag": 16,
    "Scarf": 17,
}


def get_image_dimensions(img_path):
    """
    Get the dimensions of an image.

    Args:
        img_path (str): Path to the image.

    Returns:
        tuple: (width, height) of the image.
    """
    original_image = Image.open(img_path)
    return original_image.size


def decode_base64_mask(base64_string, width, height):
    """
    Decode a base64-encoded mask into a NumPy array.

    Args:
        base64_string (str): Base64-encoded mask.
        width (int): Target width.
        height (int): Target height.

    Returns:
        np.ndarray: Single-channel mask array.
    """
    mask_data = base64.b64decode(base64_string)
    mask_image = Image.open(io.BytesIO(mask_data))
    mask_array = np.array(mask_image)
    if len(mask_array.shape) == 3:
        mask_array = mask_array[:, :, 0]  # Take first channel if RGB
    mask_image = Image.fromarray(mask_array).resize((width, height), Image.NEAREST)
    return np.array(mask_image)


def create_masks(results, width, height):
    """
    Combine multiple class masks into a single segmentation mask.

    Args:
        results (list): List of dictionaries with 'label' and 'mask' keys.
        width (int): Target width.
        height (int): Target height.

    Returns:
        np.ndarray: Combined segmentation mask with class indices.
    """
    combined_mask = np.zeros((height, width), dtype=np.uint8)  # Initialize with Background (0)

    # traitez d'abord les masques autres que ceux d'arrière-plan
    for result in results:
        label = result["label"]
        class_id = CLASS_MAPPING.get(label, 0)
        if class_id == 0:  # Skip Background
            continue
        mask_array = decode_base64_mask(result["mask"], width, height)
        combined_mask[mask_array > 0] = class_id

    # Exécutez ce processus en arrière-plan en dernier afin d'éviter de remplacer inutilement d'autres classes.
    # (Bien que le modèle fournisse généralement des masques non superposés pour les différentes classes, à l'exception du fond)
    for result in results:
        if result["label"] == "Background":
            mask_array = decode_base64_mask(result["mask"], width, height)
            # Appliquer le fond uniquement si aucune autre classe n'a été assignée
            # Cette logique pourrait nécessiter des ajustements en fonction de la définition du terme « fond » par le modèle.
            # Pour ce modèle, il semble plus sûr de laisser d'abord les autres classes écraser cette valeur.
            # Une application simple comme celle-ci devrait suffire : si le masque de fond indique que le pixel est un élément de fond, attribuer la valeur 0.
            # Cependant, une approche plus robuste consisterait à attribuer la valeur de fond uniquement si la valeur de combined_mask est encore 0 (valeur initiale).
            combined_mask[mask_array > 0] = 0  # Class ID pour le fond est 0

    return combined_mask

## 4. Segmentation d'une Seule Image

Avant de traiter toutes les images, concentrons-nous sur une seule pour bien comprendre le processus.

Étapes :
1.  Choisir une image.
2.  Ouvrir l'image en mode binaire (`"rb"`) et lire son contenu (`data`).
3.  Déterminer le `Content-Type` (par exemple, `"image/jpeg"` ou `"image/png"`).
4.  Envoyer la requête POST à l'API avec `requests.post()` en passant l'URL, les headers et les données.
5.  Vérifier le code de statut de la réponse. Une erreur sera levée si le code n'est pas 2xx (succès) grâce à `response.raise_for_status()`.
6.  Convertir la réponse JSON en un dictionnaire Python avec `response.json()`.
7.  Utiliser nos fonctions `get_image_dimensions` et `create_masks` pour obtenir le masque final.
8.  Afficher l'image originale et le masque segmenté.

In [ ]:
if image_paths:
    single_image_path = image_paths[0]  # Prenons la première image de notre liste
    print(f"Traitement de l'image : {single_image_path}")

    try:
        # Lire l'image en binaire
        # Et mettez le contenu de l'image dans la variable image_data
        with open(single_image_path, "rb") as f:
            image_data = f.read()

        # Maintenant, utilisé l'API huggingface
        # ainsi que les fonctions données plus haut pour ségmenter vos images.

        # 1) Deviner dynamiquement le Content-Type (image/jpeg, image/png, ...)
        with Image.open(single_image_path) as img:
            content_type = img.get_format_mimetype()

        # 2) Construire des headers spécifiques à cet envoi (on ajoute le Content-Type ici)
        headers_with_type = {"Authorization": f"Bearer {api_token}", "Content-Type": content_type}

        # 3) Appel à l'API Inference en POST avec l'image brute
        reponse = requests.post(API_URL, headers=headers_with_type, data=image_data, timeout=60)

        # 4) Vérifier le code de statut de la réponse. Une erreur sera levée si le code n'est pas 2xx (succès) grâce à response.raise_for_status().
        reponse.raise_for_status()

        # 5) Récupérer le JSON de sortie (liste de dicts: {'label': str, 'mask': base64})
        results = reponse.json()

        # 6) Utiliser nos fonctions get_image_dimensions et create_masks pour obtenir le masque final.
        width, height = get_image_dimensions(single_image_path)
        segmentation_mask = create_masks(results, width, height)

        # 7) obtenir l'image d'origine.
        original_image = Image.open(single_image_path)

        # 8) (Option pédagogique) Visualiser rapidement l'image et le masque
        fig = plt.figure(figsize=(12, 6))

        ax1 = plt.subplot(1, 2, 1)
        ax1.imshow(original_image)
        ax1.set_title("Image d'origine")
        ax1.axis("off")

        ax2 = plt.subplot(1, 2, 2)
        im = ax2.imshow(segmentation_mask, cmap="tab10", interpolation="nearest")
        ax2.set_title("Masque Segmenté (indices de classes)")
        ax2.axis("off")
        plt.colorbar(im, ax=ax2, fraction=0.046, pad=0.04)

       # plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Une erreur est survenue : {e}")
else:
    print("Aucune image à traiter. Vérifiez la configuration de 'image_dir' et 'max_images'.")

## 5. Segmentation de Plusieurs Images (Batch)

Maintenant que nous savons comment traiter une image, nous pouvons créer une fonction pour en traiter plusieurs.
Cette fonction va boucler sur la liste `image_paths` et appliquer la logique de segmentation à chaque image.
Nous utiliserons `tqdm` pour avoir une barre de progression.

In [ ]:
def segment_images_batch(list_of_image_paths, pause=0.5):
    """
    Segmente une liste d'images en utilisant l'API Hugging Face.

    Args:
        list_of_image_paths (list): Liste des chemins vers les images.

    Returns:
        list: Liste des masques de segmentation (tableaux NumPy).
              Contient None si une image n'a pas pu être traitée.
    """

    import time

    batch_segmentations = []
    temps_reponse = []

    for img_path in tqdm(list_of_image_paths, desc="Segmentation (HF)", unit="img"):
        try:
            # 1) Lire l'image en binaire et son contenu
            with open(img_path, "rb") as file:
                image_data = file.read()

            # 2) Deviner dynamiquement le Content-Type (image/jpeg, image/png, ...)
            with Image.open(img_path) as img:
                content_type = img.get_format_mimetype()
                # Construire des headers spécifiques à cet envoi (on ajoute le Content-Type ici)
            headers_with_type = {
                "Authorization": f"Bearer {api_token}",
                "Content-Type": content_type,
            }

            temps_debut = time.time()
            # 3) Appel API Inference et verifie le statut de la requete
            reponse = requests.post(API_URL, headers=headers_with_type, data=image_data, timeout=60)
            reponse.raise_for_status()

            temps_fin = time.time()
            duree = temps_fin - temps_debut
            temps_reponse.append(duree)

            # 4) Résultats JSON attendus: convertis en liste de dicts [{'label':..., 'mask':...}, ...]
            results = reponse.json()

            # 5) Dimensions de l'image d'origine
            width, height = get_image_dimensions(img_path)

            # 6) Masque combiné (indices de classes)
            segmentation_mask = create_masks(results, width, height)
            batch_segmentations.append(segmentation_mask)

            time.sleep(pause)

        except Exception as e:
            print(f"[!] Erreur dans le traitement de l'image {img_path} : {e}")
            batch_segmentations.append(None)

    return batch_segmentations

    # Appeler la fonction pour segmenter les images listées dans image_paths


if image_paths:
    print(f"\nTraitement de {len(image_paths)} image(s) en batch...")
    batch_seg_results = segment_images_batch(image_paths)
    print("Traitement en batch terminé.")
else:
    batch_seg_results = []
    print("Aucune image à traiter en batch.")

## 6. Affichage des Résultats en Batch

Nous allons maintenant créer une fonction pour afficher les images originales et leurs segmentations correspondantes côte à côte, dans une grille.

In [ ]:
# Colormap personnalisé pour 18 classes (valeurs 0 à 17) en RGB
custom_colormap = {
    1: (255, 255, 0),  # Jaune - Hat
    2: (255, 165, 0),  # Orange - Hair
    3: (255, 0, 255),  # Magenta - Sunglasses
    4: (255, 0, 0),  # Rouge - Upper-clothes
    5: (0, 255, 255),  # Cyan - Skirt
    6: (0, 255, 0),  # Vert - Pants
    7: (0, 0, 255),  # Bleu - Dress
    8: (128, 0, 128),  # Violet - Belt
    9: (255, 255, 0),  # Jaune - Left-shoe
    10: (255, 140, 0),  # Orange foncé - Right-shoe
    11: (140, 180, 200),  # Beige - Face
    12: (140, 180, 200),  # Beige - Left-leg
    13: (140, 180, 200),  # Beige - Right-leg
    14: (140, 180, 200),  # Beige - Left-arm
    15: (140, 180, 200),  # Beige - Right-arm
    16: (0, 128, 255),  # Bleu clair - Bag
    17: (255, 20, 147),  # Rose - Scarf
}

# Légendes associées aux labels
legend_labels = {
    "0": "Background",
    "1": "Hat",
    "2": "Hair",
    "3": "Sunglasses",
    "4": "Upper-clothes",
    "5": "Skirt",
    "6": "Pants",
    "7": "Dress",
    "8": "Belt",
    "9": "Left-shoe",
    "10": "Right-shoe",
    "11": "Face",
    "12": "Left-leg",
    "13": "Right-leg",
    "14": "Left-arm",
    "15": "Right-arm",
    "16": "Bag",
    "17": "Scarf",
}


def colorize_mask(mask, colormap):
    """
    Applique le colormap personnalisé au masque.
    Pour chaque pixel, s'il correspond à un label défini dans colormap,
    la couleur correspondante est assignée.
    """
    colored_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    for label, color in colormap.items():
        colored_mask[mask == label] = color
    return colored_mask


def display_segmented_images_batch_color(
    original_image_paths, segmentation_masks, original_mask_paths, colors, labels
):
    """
    Affiche les images originales et les masques originales et segmentés.

    Args:
        original_image_paths (list): Liste des chemins des images originales.
        original_mask_paths (list): Liste des chemins des masques originales.
        segmentation_masks (list): Liste des masques segmentés (NumPy arrays).
        colors - colormap
        labels - legend labels
    """

    nb_cols = 3
    nb_images = len(original_image_paths)
    if nb_images == 0:
        print("Aucune image à afficher.")
        return

    # on créé une figure : 4 pouces de hauteur par image, 12 pouces de largeur au total, 3 colonnes (Original | Masque Segmenté  | Masque Original ) par image
    fig, axes = plt.subplots(
        nb_images, nb_cols, figsize=(12, 4 * nb_images), squeeze=False, constrained_layout=True
    )

    for i, (img_path, mask_seg, mask_orig) in enumerate(
        zip(original_image_paths, segmentation_masks, original_mask_paths, strict=False)
    ):
        if mask_seg is None:
            continue  # Ignore les images avec masque non généré

        # --- Colonne 1 : Image originale
        img = Image.open(img_path)
        axes[i, 0].imshow(img)
        axes[i, 0].set_title(f"Original\n{Path(img_path).name}", fontsize=10)
        axes[i, 0].axis("off")

        # Masque segmenté
        colored_mask = colorize_mask(mask_seg, colors)
        ax_seg = axes[i, 1]
        ax_seg.imshow(colored_mask)
        ax_seg.set_title("Segmentation", fontsize=10)
        ax_seg.axis("off")
        ## Trouver quelles classes sont présentes dans ce masque
        classes_in_mask = np.unique(mask_seg)
        ## Préparer les patches pour ces classes
        patches = []
        for c in classes_in_mask:
            if c == 0:  # ignore background
                continue
            color = colors.get(c, (0, 0, 0))
            color_norm = (color[0] / 255, color[1] / 255, color[2] / 255)
            patch = mpatches.Patch(color=color_norm, label=labels.get(str(c), f"Class {c}"))
            patches.append(patch)
        # Afficher la légende localement dans la 2e colonne (masques segmentés)
        ax_seg.legend(
            handles=patches, loc="upper left", labelcolor="white", fontsize="x-small", frameon=False
        )

        # Masque d'origine
        msk = Image.open(mask_orig)
        msk = np.array(msk, dtype=np.uint8)
        colored_mask_orig = colorize_mask(msk, colors)
        ax_seg = axes[i, 2]
        ax_seg.imshow(colored_mask_orig)
        ax_seg.set_title("Masque d Origine")
        ax_seg.axis("off")
        ## Trouver quelles classes sont présentes dans ce masque
        classes_in_mask_orig = np.unique(msk)
        ## Préparer les patches pour ces classes
        patches = []
        for c in classes_in_mask_orig:
            if c == 0:  # ignore background
                continue
            color = colors.get(c, (0, 0, 0))
            color_norm = (color[0] / 255, color[1] / 255, color[2] / 255)
            patch = mpatches.Patch(color=color_norm, label=labels.get(str(c), f"Class {c}"))
            patches.append(patch)
        ## Afficher la légende localement dans la 3e colonne (masques d'origine)
        ax_seg.legend(
            handles=patches, loc="upper left", labelcolor="white", fontsize="x-small", frameon=False
        )

    # plt.tight_layout()
    plt.show()


# Afficher les résultats du batch
if batch_seg_results:
    display_segmented_images_batch_color(
        image_paths, batch_seg_results, mask_paths, custom_colormap, legend_labels
    )
else:
    print("Aucun résultat de segmentation à afficher.")

## 7. Calcul des métriques

Faisons un chiffrage approximatif du coût d’utilisation de cette solution avec l’API de Hugging Face. Une évaluation du coût pour 500 000 images sur 30 jours sera faite

In [ ]:
from typing import Dict
from sklearn.metrics import confusion_matrix

""" Calculate metrics for model evaluation

 y_true and y_pred: flattened arrays of pixel labels
 num_classes = 18
"""

def segmentation_metrics(mask_true, mask_pred, num_classes=18) -> Dict[str, float]:

    mask_true = np.array(mask_true).flatten()
    mask_pred = np.array(mask_pred).flatten()
    
    cm = confusion_matrix(mask_true, mask_pred, labels=np.arange(num_classes))
    TP = np.diag(cm)
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (FP + FN + TP)

    pixel_acc = TP.sum() / cm.sum()
    mean_acc = np.mean(TP / (TP + FN + 1e-10))
    iou = TP / (TP + FP + FN + 1e-10)
    mean_iou = np.mean(iou)
    dice = 2*TP / (2*TP + FP + FN + 1e-10)
    precision = TP / (TP + FP + 1e-10)
    recall = TP / (TP + FN + 1e-10)

    return {
        "Pixel Acc": pixel_acc,
        "Mean Acc": mean_acc,
        "mIoU": mean_iou,
        "Dice": dice.mean(),
        "Per-class IoU": iou,
        "Per-class Precision": precision,
        "Per-class Recall": recall
    }

iou_per_image = []

if batch_seg_results:
    
    for i, (mask_orig, mask_seg) in enumerate(zip(mask_paths, batch_seg_results)):
        if mask_seg is None:
            continue  # Ignore les images avec masque non généré
            
        # masque orig
        msk = Image.open(mask_orig)  
        msk_array = np.array(msk)
        
        metrics = segmentation_metrics(msk_array, mask_seg, num_classes=18)
        #print(f"Image {i}: {metrics}")
        iou_dict = {f"class_{c}": iou_val for c, iou_val in enumerate(metrics["Per-class IoU"])}

        # Ajout du numéro d'image
        iou_dict["image_index"] = i
        iou_per_image.append(iou_dict)    

    # Conversion en DataFrame pandas
    df_iou = pd.DataFrame(iou_per_image)
    df_iou.set_index("image_index", inplace=True)
    
    print(f"Mean IoU \n{df_iou.mean().round(2)}")
    
    df_iou_nonzero = df_iou.replace(0, np.nan)
    # Calcul de la moyenne par classe en excluant les zéros
    mean_iou_nonzero = df_iou_nonzero.mean(skipna=True)
    print(f"Mean IoU without zeros \n{mean_iou_nonzero.round(2)}")
    print(f"Mean IoU of all clases without zeros \n{mean_iou_nonzero.mean():.2f}")
    
else:
    print("Aucun résultat de segmentation à afficher.")


In [ ]:
from dataclasses import dataclass


@dataclass
class Workload:
    n_images: int = 500_000  # nombre total d'images
    days: int = 30  # période d'étude (jours)
    hours_per_day: float = 24.0  # fenêtre quotidienne d'activité (24 = fonctionnement continu)
    ips: float | None = (
        None  # images par seconde EFFECTIVES (si connu); si None, on ne calcule pas l'heure active minimale
    )


@dataclass
class EndpointPlan:
    name: str
    hourly_rate_usd: float  # coût $/h de l'instance (doc HF)
    min_replicas: int = 1  # réplicas minimum toujours allumés (autoscaling min)
    extra_replicas: int = 0  # réplicas additionnels pendant les pics (optionnel)
    extra_hours_per_day: float = (
        0.0  # nombre d'heures/jour où les réplicas additionnels sont actifs
    )


def total_hours(days: int, hours_per_day: float) -> float:
    return days * hours_per_day


def cost_flat(plan: EndpointPlan, days: int, hours_per_day: float) -> float:
    """
    Coût si on garde les réplicas min allumés toute la fenêtre, avec éventuellement des réplicas
    additionnels sur une fraction du temps (ex: pics).
    """
    base = plan.hourly_rate_usd * plan.min_replicas * total_hours(days, hours_per_day)
    extra = plan.hourly_rate_usd * plan.extra_replicas * (days * plan.extra_hours_per_day)
    return base + extra


def min_active_hours_needed(n_images: int, ips: float) -> float:
    """
    Heures strictement nécessaires pour traiter n_images à un débit ips.
    """
    if ips <= 0:
        raise ValueError("ips doit être > 0")
    seconds = n_images / ips
    return seconds / 3600.0


def cost_scale_to_zero_like(plan: EndpointPlan, needed_hours: float) -> float:
    """
    Approximation d'un endpoint qui ne tourne que le temps 'utile' (scale-to-zero).
    Dans la réalité, prévois une marge (warmup, files d’attente).
    """
    return plan.hourly_rate_usd * plan.min_replicas * needed_hours


# ---------- Scénarios prêts à l'emploi ----------

wl = Workload(n_images=500_000, days=30, hours_per_day=24.0)

cpu_small = EndpointPlan(name="CPU aws intel-spr x2", hourly_rate_usd=0.067, min_replicas=1)
gpu_t4 = EndpointPlan(name="GPU aws nvidia-t4 x1", hourly_rate_usd=0.5, min_replicas=1)

# 1) Fonctionnement CONTINU (24/7) sur 30 jours (approche simple, sans supposer un débit)
cost_cpu_flat = cost_flat(cpu_small, wl.days, wl.hours_per_day)  # ~ 0.067 * 30 * 24
cost_gpu_flat = cost_flat(gpu_t4, wl.days, wl.hours_per_day)  # ~ 0.5   * 30 * 24

print(f"[Flat 24/7] CPU intel-spr x2 (1 réplique): ${cost_cpu_flat:,.2f}")
print(f"[Flat 24/7] GPU nvidia-t4 x1 (1 réplique): ${cost_gpu_flat:,.2f}")

# 2) Si tu connais un DÉBIT effectif (ips), on calcule le temps minimal 'utile'
#    Exemples de débits illustratifs (à ajuster selon tes mesures réelles):
for assumed_ips in [0.5, 1.0, 2.0, 5.0]:
    needed_h = min_active_hours_needed(wl.n_images, assumed_ips)
    cpu_cost_scaled = cost_scale_to_zero_like(cpu_small, needed_h)
    gpu_cost_scaled = cost_scale_to_zero_like(gpu_t4, needed_h)
    print(f"\n[Scale-to-zero approx] {assumed_ips} img/s")
    print(f"Heures actives nécessaires ≈ {needed_h:.1f} h")
    print(f"Coût CPU intel-spr x2 (1 réplique): ${cpu_cost_scaled:,.2f}")
    print(f"Coût GPU nvidia-t4 x1 (1 réplique): ${gpu_cost_scaled:,.2f}")

# 3) Exemple 'pics' : 1 réplique min 24/7 + 2 réplicas additionnels 2 h/jour (trafic concentré)
cpu_bursty = EndpointPlan(
    name="CPU bursty",
    hourly_rate_usd=0.067,
    min_replicas=1,
    extra_replicas=2,
    extra_hours_per_day=2.0,
)
print(
    f"\n[Bursty] CPU 1 réplique 24/7 + 2 réplicas pendant 2h/jour sur 30j : ${cost_flat(cpu_bursty, wl.days, wl.hours_per_day):,.2f}"
)

# Photo instantanée : combien coûte un mois en continu pour chaque plan.
labels = ["CPU 24/7", "GPU 24/7"]
values = [cost_cpu_flat, cost_gpu_flat]

plt.figure(figsize=(6, 4))
plt.bar(labels, values)
plt.ylabel("Coût total (USD) sur 30 jours")
plt.title("Comparaison des coûts 24/7")
for i, v in enumerate(values):
    plt.text(i, v, f"${v:,.0f}", ha="center", va="bottom")
plt.tight_layout()
plt.show()

## Conclusion et Prochaines Étapes

Félicitations ! Vous avez appris à :
- Configurer les appels à l'API d'inférence Hugging Face.
- Envoyer des images pour la segmentation.
- Interpréter les résultats (avec l'aide des fonctions fournies).
- Visualiser les segmentations.

Pistes d'amélioration ou d'exploration :
- **Gestion d'erreurs plus fine** : Implémenter des tentatives multiples (retry) en cas d'échec de l'API (par exemple, si le modèle est en cours de chargement).
- **Appels asynchrones** : Pour un grand nombre d'images, des appels asynchrones (avec `asyncio` et `aiohttp`) seraient beaucoup plus rapides.
- **Autres modèles** : Explorer d'autres modèles de segmentation ou d'autres tâches sur Hugging Face Hub.

N'hésitez pas à modifier le code, à tester avec vos propres images et à explorer davantage !

**_Note_** : Si vous aimez ce modèle, n'hésitez pas à le [télécharger](https://huggingface.co/sayeed99/segformer_b3_clothes) et jouer avec directement sur votre machine !